In [54]:
import os
import sys
from pathlib import Path
import sqlite3 as sql
import pandas as pd

In [55]:
BASE_DIR = Path.cwd().parent
DB_PATH = BASE_DIR / "data" / "portugal_squad.db"
SRC_DIR = BASE_DIR / "src"
sys.path.append(str(BASE_DIR))
# print(f"Project Root: {BASE_DIR}")
# print(f"Database Path: {DB_PATH}")
# print(f"src path: {SRC_DIR}")

In [56]:
# MAKE sql connection
conn = sql.connect(DB_PATH)
query_forward = 'SELECT * FROM forward_base'
query_mid = 'SELECT * FROM midfield_base'
query_def = 'SELECT * FROM defense_base'
query_gk  = 'SELECT * FROM goalkeeper_base'


In [57]:
df_fw = pd.read_sql_query(query_forward,conn)
df_mid = pd.read_sql_query(query_mid,conn)
df_def = pd.read_sql_query(query_def,conn)
df_gk = pd.read_sql_query(query_gk,conn)
conn.close()


In [58]:
df_fw.head(10)

,name,season_name,position,stat_name,per_90
0,Cristiano Ronaldo,2024 UEFA Euro 2024,FW-MF (WM),npxG: Non-Penalty xG,0.52
1,Cristiano Ronaldo,2024 UEFA Euro 2024,FW-MF (WM),Shots Total,4.08
2,Cristiano Ronaldo,2024 UEFA Euro 2024,FW-MF (WM),xAG: Exp. Assisted Goals,0.22
3,Cristiano Ronaldo,2024 UEFA Euro 2024,FW-MF (WM),Shot-Creating Actions,2.41
4,Cristiano Ronaldo,2024 UEFA Euro 2024,FW-MF (WM),Successful Take-Ons,0.56
5,Cristiano Ronaldo,2024 UEFA Euro 2024,FW-MF (WM),Touches (Att Pen),4.45
6,Cristiano Ronaldo,2024 UEFA Euro 2024,FW-MF (WM),Progressive Passes Rec,6.12
7,Cristiano Ronaldo,2022-2023 Europa League,FW-MF (WM),npxG: Non-Penalty xG,0.68
8,Cristiano Ronaldo,2022-2023 Europa League,FW-MF (WM),Shots Total,5.09
9,Cristiano Ronaldo,2022-2023 Europa League,FW-MF (WM),xAG: Exp. Assisted Goals,0.29


In [59]:
df_mid.head()

,name,season_name,position,stat_name,per_90
0,Vitinha,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",MF (CM-DM),xAG: Exp. Assisted Goals,0.13
1,Vitinha,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",MF (CM-DM),Shot-Creating Actions,3.95
2,Vitinha,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",MF (CM-DM),Pass Completion %,93.4%
3,Vitinha,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",MF (CM-DM),Progressive Passes,9.23
4,Vitinha,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",MF (CM-DM),Progressive Carries,2.58


In [60]:
df_def.head()

,name,season_name,position,stat_name,per_90
0,Renato Veiga,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",DF-MF,Assists,0.03
1,Renato Veiga,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",DF-MF,Progressive Passes,4.06
2,Renato Veiga,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",DF-MF,Tackles,0.88
3,Renato Veiga,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",DF-MF,Interceptions,1.02
4,Renato Veiga,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",DF-MF,Blocks,1.41


In [61]:
df_wide = (
    df_def
    .pivot_table(
        index=["name", "season_name", "position"],
        columns="stat_name",
        values="per_90"
    )
    .reset_index()
)


df_wide.columns.name = None

df_wide.head()


,name,season_name,position,Aerials Won,Assists,Blocks,Clearances,Interceptions,Progressive Passes,Tackles
0,António Silva,2022-2023 Champions League,DF (CB,1.01,0.0,1.31,3.42,1.41,3.02,1.11
1,António Silva,2022-2023 Primeira Liga,DF (CB,1.54,0.0,0.77,2.98,1.27,4.01,1.91
2,António Silva,2023-2024 Champions League,DF (CB,0.97,0.0,0.97,5.20,0.32,1.30,2.60
3,António Silva,2023-2024 Europa League,DF (CB,0.95,0.0,1.58,5.53,0.63,4.26,1.74
4,António Silva,2023-2024 Primeira Liga,DF (CB,1.79,0.0,1.62,3.13,0.79,4.61,2.20


In [ ]:
# create a function to select the euro 2024 as our primary metric target
season_name = 'Euro 2024'

def select_a_season(df:pd.DataFrame,season_name:str)-> pd.DataFrame:
    df['priority'] = df['season_name'].apply(lambda x: 1 if season_name in x else 2)
    print(df.head())
    # first pivot the table
    df_wide = df.pivot_table(index=['name','season_name','position','priority'],columns='stat_name',values='per_90').reset_index()
    df_wide.columns.name = None
    return df_wide
    df_wide  = df_wide[df_wide['priority']==1]

    return df_wide



In [65]:
df_fw_euro = select_a_season(df_fw,'Euro 2024')
df_fw_euro.head(109)

,name,season_name,position,priority,Progressive Passes Rec,Shot-Creating Actions,Shots Total,Successful Take-Ons,Touches (Att Pen),npxG: Non-Penalty xG,xAG: Exp. Assisted Goals
0,Carlos Forbs,2023-2024 Eredivisie,FW-MF,2,13.03,3.80,2.06,2.06,6.62,0.23,0.23
1,Carlos Forbs,2023-2024 Europa League,FW-MF,2,9.58,3.03,0.76,0.76,3.28,0.30,0.35
2,Carlos Forbs,2025-2026 Belgian Pro League,FW-MF,2,15.72,4.93,1.55,1.41,6.55,0.16,0.21
3,Carlos Forbs,2025-2026 Champions League,FW-MF,2,7.45,4.60,1.97,0.66,4.82,0.37,0.42
4,Carlos Forbs,Last 365 Days Men's Next 14 Competitions,FW-MF,2,15.72,4.93,1.55,1.41,6.55,0.16,0.21
...,...,...,...,...,...,...,...,...,...,...,...
70,Rafael Leão,2024 UEFA Euro 2024,FW-MF (AM,1,10.03,3.14,0.94,2.20,5.33,0.06,0.16
71,Rafael Leão,2024-2025 Champions League,FW-MF (AM,2,12.03,3.40,1.82,4.43,6.92,0.25,0.17
72,Rafael Leão,2024-2025 Serie A,FW-MF (AM,2,10.15,4.38,2.56,2.29,5.89,0.29,0.33
73,Rafael Leão,2025-2026 Serie A,FW-MF (AM,2,5.74,2.43,3.09,0.77,5.74,0.40,0.24


In [66]:
df_mid.head()

,name,season_name,position,stat_name,per_90
0,Vitinha,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",MF (CM-DM),xAG: Exp. Assisted Goals,0.13
1,Vitinha,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",MF (CM-DM),Shot-Creating Actions,3.95
2,Vitinha,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",MF (CM-DM),Pass Completion %,93.4%
3,Vitinha,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",MF (CM-DM),Progressive Passes,9.23
4,Vitinha,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",MF (CM-DM),Progressive Carries,2.58


In [67]:
df_mid_euro = select_a_season(df_mid,'Euro 2024')
df_mid_euro.head()

TypeError: agg function failed [how->mean,dtype->object]

In [27]:
df_def_euro = select_a_season(df_def,'Euro 2024')
df_def_euro.head()

,name,season_name,position,priority,Aerials Won,Assists,Blocks,Clearances,Interceptions,Progressive Passes,Tackles
90,Rúben Dias,2024 UEFA Euro 2024,DF-MF (CB),1,2.36,0.0,1.5,2.57,0.64,4.71,0.43


In [28]:
df_mid.head()

,name,season_name,position,stat_name,per_90,priority
0,Vitinha,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",MF (CM-DM),xAG: Exp. Assisted Goals,0.13,2
1,Vitinha,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",MF (CM-DM),Shot-Creating Actions,3.95,2
2,Vitinha,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",MF (CM-DM),Pass Completion %,93.4%,2
3,Vitinha,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",MF (CM-DM),Progressive Passes,9.23,2
4,Vitinha,"Last 365 Days Men's Big 5 Leagues, UCL, UEL",MF (CM-DM),Progressive Carries,2.58,2
